In [1]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from scipy.stats import pearsonr
from tqdm import tqdm
from scipy import ndimage
from IPython.display import clear_output

Using TensorFlow backend.


In [3]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 32
NUM_CLASSES = 10
NUM_CHANNELS = 3
MODEL_NAME = "CIFAR_rotation"
VOTING = 'HARD'
PATH = "/content/drive/My Drive/NC/"
MODEL_ADDITION_DELTA = 0.01
MODEL_ADDITION_PATIENCE = 3
NR_OF_RUNS = 10

# Preprocess

In [4]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [5]:
(x_train_val, y_train_val), (x_test, y_test) = cifar10.load_data()

#x_train_val = preprocess(x_train_val)
#x_test = preprocess(x_test)

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [7]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [8]:
def CIFARmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=2)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters=10, kernel_size=(1,1),strides=(1,1), padding='valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

#Predict


In [9]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y,voting = 'hard'):
    
    if voting == "soft":
      prediction = soft_voting(models, X)
    elif voting == "hard":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")

    return accuracy_score(prediction, np.argmax(Y, axis=1))

#Augmentation 


In [10]:
def rotate_image(image):
    if np.random.rand() < 0.2:
        angles = np.linspace(1,10,10)
        rotation_angle = np.random.choice(angles)
        if np.random.rand() < 0.5:
            image = ndimage.rotate(image, rotation_angle, reshape = False)
        else:
            image = ndimage.rotate(image, -rotation_angle, reshape = False) 
    return image

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Train

In [13]:
for run in range(1, NR_OF_RUNS+1):

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    models = []
    accuracies = [0]
    patience = 0

    for i in count(1):

        print(f"\n ===== Train model {i} =====")

        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # augmentation
        datagen = ImageDataGenerator(preprocessing_function=rotate_image)
        datagen = datagen.flow(x_train,y_train, batch_size= BATCH_SIZE)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)

        # Create the model
        model = CIFARmodel(IMAGE_SIZE, NUM_CLASSES, 3)
        
        # Load the weighs if the model is already trained
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{i}.h5"

        if os.path.exists(weights_path):
            print(f"Skipping training of model {i}: weights exists")
            model.load_weights(weights_path)
        else:
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{i}.csv", separator=';')

            model.fit_generator(datagen,
                                epochs = EPOCHS,
                                validation_data = (x_val, y_val),
                                shuffle = True,
                                callbacks=[es, csv_logger])
            
            model.save_weights(weights_path)
        
        models.append(model)

        acc = predict(models, x_val, y_val)
        delta = acc - accuracies[-1]

        accuracies.append(acc)

        if delta >= MODEL_ADDITION_DELTA:
          patience = 0
        else:
          patience += 1

        print(f"Model: {i} added. Resulting score: {acc}, Delta: {delta}, Patience: {patience}")

        if patience >= MODEL_ADDITION_PATIENCE:
          break
    print("\n ===== Accuracy vs nr of models =====")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy_{VOTING}.csv")
    display(accuracy_df)

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(models, x_test, y_testc,voting='hard')
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("\n ===== Correlation =====")
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in predictions:
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix_{VOTING}.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + f"/results_{VOTING}.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)
    # Results

    ## Accuracy vs nr of models
    ## Visualizing the accuracy vs the number of models in the ensamble

    print("\n ===== Accuracy vs nr of models =====")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy_SOFT.csv")
    display(accuracy_df)

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(models, x_test, y_testc,voting='soft')
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("\n ===== Correlation =====")
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in predictions:
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix_SOFT.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + f"/results_SOFT.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)



 ===== Train model 1 =====
Epoch 1/9999
313/313 [==============================] - 17s 55ms/step - loss: 1.7164 - accuracy: 0.3900 - val_loss: 2.2370 - val_accuracy: 0.1460
Epoch 2/9999
313/313 [==============================] - 11s 34ms/step - loss: 1.3543 - accuracy: 0.5191 - val_loss: 1.4246 - val_accuracy: 0.5059
Epoch 3/9999
313/313 [==============================] - 11s 34ms/step - loss: 1.2052 - accuracy: 0.5743 - val_loss: 1.1977 - val_accuracy: 0.5751
Epoch 4/9999
313/313 [==============================] - 10s 34ms/step - loss: 1.1026 - accuracy: 0.6122 - val_loss: 1.1686 - val_accuracy: 0.5857
Epoch 5/9999
313/313 [==============================] - 11s 34ms/step - loss: 1.0173 - accuracy: 0.6463 - val_loss: 1.0656 - val_accuracy: 0.6221
Epoch 6/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.9457 - accuracy: 0.6725 - val_loss: 1.0874 - val_accuracy: 0.6186
Epoch 7/9999
313/313 [==============================] - 11s 34ms/step - loss: 0.8848 - accuracy:

,Accuracy,Nr of models
0,0.0000,0
1,0.6514,1
2,0.6716,2
3,0.7058,3
4,0.7056,4
5,0.7242,5
6,0.7260,6
7,0.7299,7
8,0.7321,8



 ===== Accuracy ======
Accuracy: 0.7326

 ===== Correlation =====


,0,1,2,3,4,5,6,7
0,NaN,0.466120,0.479412,0.451918,0.449131,0.418505,0.523194,0.479315
1,0.466120,NaN,0.462931,0.481331,0.512038,0.511079,0.494665,0.462872
2,0.479412,0.462931,NaN,0.491552,0.498492,0.488470,0.480992,0.485733
3,0.451918,0.481331,0.491552,NaN,0.467583,0.479603,0.485243,0.462018
4,0.449131,0.512038,0.498492,0.467583,NaN,0.506749,0.487452,0.496760
5,0.418505,0.511079,0.488470,0.479603,0.506749,NaN,0.447035,0.482669
6,0.523194,0.494665,0.480992,0.485243,0.487452,0.447035,NaN,0.482478
7,0.479315,0.462872,0.485733,0.462018,0.496760,0.482669,0.482478,NaN


Average correlation: 0.4798336092081428

 ===== Accuracy vs nr of models =====


...

Average correlation: 0.46347952924783475

 ===== Accuracy vs nr of models =====


,Accuracy,Nr of models
0,0.0000,0
1,0.6115,1
2,0.6703,2
3,0.6945,3
4,0.7095,4
5,0.7179,5
6,0.7161,6
7,0.7278,7
8,0.7293,8
9,0.7319,9



 ===== Accuracy ======
Accuracy: 0.7324

 ===== Correlation =====


,0,1,2,3,4,5,6,7,8,9
0,NaN,0.489736,0.515810,0.443790,0.383571,0.426978,0.362346,0.377194,0.450910,0.448354
1,0.489736,NaN,0.523714,0.536010,0.471008,0.484681,0.446999,0.449055,0.466727,0.519923
2,0.515810,0.523714,NaN,0.508679,0.399884,0.501216,0.443723,0.387742,0.498121,0.478327
3,0.443790,0.536010,0.508679,NaN,0.511959,0.481465,0.507889,0.468721,0.441858,0.527653
4,0.383571,0.471008,0.399884,0.511959,NaN,0.461557,0.510675,0.484349,0.412273,0.515630
5,0.426978,0.484681,0.501216,0.481465,0.461557,NaN,0.490632,0.432734,0.537527,0.441674
6,0.362346,0.446999,0.443723,0.507889,0.510675,0.490632,NaN,0.421750,0.440706,0.479628
7,0.377194,0.449055,0.387742,0.468721,0.484349,0.432734,0.421750,NaN,0.365384,0.479509
8,0.450910,0.466727,0.498121,0.441858,0.412273,0.537527,0.440706,0.365384,NaN,0.428511
9,0.448354,0.519923,0.478327,0.527653,0.515630,0.441674,0.479628,0.479509,0.428511,NaN


Average correlation: 0.46347952924783475


# Results

In [14]:
from scipy import stats

In [15]:
baseline_hard =  [0.7261,
0.7174,
0.7199,
0.7196,
0.7185,
0.7201,
0.7163,
0.7107,
0.72,
0.7199]

In [16]:
np.mean(baseline_hard)

0.71885

In [17]:
baseline_soft=   [0.7311,
0.7262,
0.7252,
0.7204,
0.7207,
0.7208,
0.7273,
0.7228,
0.7233,
0.7301]

In [18]:
np.mean(baseline_soft)

0.7247899999999999

In [24]:
augmented_hard = pd.read_csv(PATH+ "CIFAR_rotation/results_HARD.csv")
augmented_hard = augmented_hard.values
augmented_hard

array([[ 1.        ,  0.7326    ,  0.47983361],
       [ 2.        ,  0.7313    ,  0.47109997],
       [ 3.        ,  0.7203    ,  0.4783455 ],
       [ 4.        ,  0.728     ,  0.48827692],
       [ 5.        ,  0.7174    ,  0.47138347],
       [ 6.        ,  0.7188    ,  0.48603817],
       [ 7.        ,  0.7201    ,  0.47936342],
       [ 8.        ,  0.7258    ,  0.47636431],
       [ 9.        ,  0.725     ,  0.47767154],
       [10.        ,  0.7279    ,  0.46347953]])

In [25]:
augmented_soft = pd.read_csv(PATH+"CIFAR_rotation/results_SOFT.csv")
augmented_soft = augmented_soft.values
augmented_soft

array([[ 1.        ,  0.7411    ,  0.47983361],
       [ 2.        ,  0.7389    ,  0.47109997],
       [ 3.        ,  0.7301    ,  0.4783455 ],
       [ 4.        ,  0.7385    ,  0.48827692],
       [ 5.        ,  0.7303    ,  0.47138347],
       [ 6.        ,  0.73      ,  0.48603817],
       [ 7.        ,  0.7286    ,  0.47936342],
       [ 8.        ,  0.7372    ,  0.47636431],
       [ 9.        ,  0.7338    ,  0.47767154],
       [10.        ,  0.7324    ,  0.46347953]])

In [26]:
augmented_hard = augmented_hard[:,1]
augmented_soft = augmented_soft[:,1]

In [27]:
np.mean(augmented_hard)

0.7247199999999999

In [28]:
np.mean(augmented_soft)

0.7340899999999999

In [29]:
stats.ttest_rel(augmented_hard,baseline_hard)

Ttest_relResult(statistic=3.236995628815524, pvalue=0.010207640998461252)

In [30]:
stats.ttest_rel(augmented_soft,baseline_soft)

Ttest_relResult(statistic=5.586708076956365, pvalue=0.0003400133228561551)

## Accuracy
The final accuracy of the ensamble on the test set

In [ ]:
!zip -r /content/FashionMNIST_rotation.zip /content/FashionMNIST_rotation

  adding: content/FashionMNIST_rotation/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/accuracy_SOFT.csv (deflated 38%)
  adding: content/FashionMNIST_rotation/10/weights/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/weights/weights-1.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-5.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-2.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-6.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-4.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-3.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/history/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/history/history-4.csv (deflated 52%)
  adding: content/FashionMNIST_rotation/10/history/history-1.csv (deflated 52%)
  adding: content/FashionMNIST_rotation/10/history/hi